In [1]:
import pandas
%reset -f

import sys
sys.path.append("../data/exadata/parquet_dataset/query_tool")

from toolz import pipe
import pandas as pd
import numpy as np
import datetime
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.linear_model import LinearRegression
from query_tool import M100DataClient
from matplotlib import pyplot as plt
from models import LinearNN, StateEstimator

dataset_path = "../data/m100"
client = M100DataClient(dataset_path)



In [2]:
"""Cell purpose: pull desired features from plugins"""


#TODO: Merging method, working for columns that aren't timestamp,node,value
#For now I think it's necessary to do it this way because of memory issues
def extract_features(feature_list, n_rows, columns, year_month):
    df = pd.DataFrame()
    for f in feature_list: 
        query_df = client.query(f, 
                      columns=columns,
                      year_month=year_month)
        
        samples = np.random.randint(low = 0, high = query_df.shape[0], size = n_rows)
        sampled_df = query_df if query_df.shape[0] < n_rows else query_df.iloc[samples] 
        multi_idx= pd.MultiIndex.from_arrays([sampled_df["timestamp"].values, sampled_df["node"].values], names = ("timestamp", "node"))
        data = {f: sampled_df["value"].values}
        processed_df = pd.DataFrame(index = multi_idx, data=data)
        if df.empty:
            df = processed_df
        else:
            # df = pd.merge(df, processed_df, left_index=True, right_index=True)
            # df = pd.merge(df, processed_df, how="outer")
            df = pd.concat([df, processed_df], join="outer")
    return df

n_rows = 50
year_month = ["22-02"]

#ipmi collects physical/hardware properties of nodes
ipmi_features = ["ambient", "gpu0_core_temp", "p0_core0_temp"]
ipmi_columns = ["timestamp", "node", "value"]
ipmi_df = extract_features(ipmi_features, n_rows, ipmi_columns, year_month)


#vertiv plugin collects data for AC cooling 
# vertiv_df = client.query(["Supply_Air_Temperature", "Return_Air_Temperature"],
#                   columns=['timestamp','value'],
#                   year_month=["22-02"])


# #ganglia collects metrics on utilization of hardware
# ganglia_df = client.query(["load_fifteen"],
#                   columns=['timestamp', "node", 'value'],
#                   year_month=["22-02"])

# logics collects power consumption data from equipment and devices
# logics_df = client.query(["Tot"],
#                   columns=['timestamp', "device", "panel", 'value'],
#                   year_month=["22-02"])

#schneider collects data on liquid cooling system
# schneider_df = client.query(["Tot"],
#                   columns=['timestamp', "node", 'value'],
#                   year_month=["22-02"])

#weather data
# weather_df = client.query(["temp"],
#                   columns=['timestamp', 'value'],
#                   year_month=["22-02"])


ipmi_df.head(100)


Retrieving data of type: float
Retrieving data of type: int32
Retrieving data of type: int32


,,ambient,gpu0_core_temp,p0_core0_temp
timestamp,node,,,
2022-02-23 08:39:40,73,23.600000,<NA>,<NA>
2022-02-20 10:15:00,467,21.200001,<NA>,<NA>
2022-02-09 22:48:20,479,26.000000,<NA>,<NA>
2022-02-13 13:06:20,680,19.200001,<NA>,<NA>
2022-02-01 06:18:40,812,23.799999,<NA>,<NA>
...,...,...,...,...
2022-02-20 03:43:20,189,NaN,67,<NA>
2022-02-18 12:40:20,74,NaN,43,<NA>
2022-02-05 16:49:20,602,NaN,31,<NA>


In [3]:
# dfs = [weather_df, schneider_df, logics_df, ganglia_df, ipmi_df, vertiv_df]
# plugin_dfs = [ipmi_df, vertiv_df, weather_df]
plugin_dfs = [ipmi_df]
for d in plugin_dfs:
    print(d.shape)
    print(d.pivot(index="timestamp").head(3))

(150, 3)


TypeError: pivot() missing 1 required keyword-only argument: 'columns'

In [ ]:
"""Cell purpose: preprocess and/or merge data as desired"""
df = pd.concat(plugin_dfs)
# print(merged_df["timestamp"].astype(np.int64) * 10e-9)
# merge_df["timestamp"] = merge_df["timestamp"].apply(lambda ts: ts.as_unit("ns"))
df.head(10)

In [ ]:
# Will have to make work with timestamp indexign later
class M100Data(Dataset):
    def __init__(self, df, transform = None, label_transform = None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df.shape[0])

    def __getitem__(self, idx):
        pass
        # return feat, label
    
    

In [ ]:
trainloader = DataLoader(train_ds, batch_size = 8, shuffle = True)
model = LinearNN()

In [ ]:
"""Cell purpose: train"""
# model = LinearRegression().fit(merged_df["air_temp"].to_numpy().reshape(-1,1), merged_df["ambient_temp"].to_numpy().reshape(-1,1))

In [ ]:
"""Cell purpose: inference"""
# preds = model.predict(test_df["air_temp"].to_numpy().reshape(-1,1))


In [ ]:
"""Cell purpose: benchmarking"""

# labels = test_df["ambient_temp"].to_numpy().reshape(-1,1)
# 
# 
# timesteps = [t for t in range(preds.shape[0])]
# colors = iter(plt.cm.rainbow(np.linspace(0, 0.5, 2)))
# plt.scatter(preds, labels, c=["blue", "red"])
# 
# plt.legend(loc="upper left")
# plt.xlabel("Time")
# plt.ylabel("Ambient temperature")
# plt.grid()
# plt.show()



In [ ]:
""
